<a href="https://colab.research.google.com/github/MelDashti/Smart-Chatbot/blob/master/AIChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/MelDashti/Smart-Chatbot.git

Cloning into 'Smart-Chatbot'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (6/6), done.


In [ ]:
import os
os.chdir('/content/') # Here we set the working directory

import pandas as pd
df = pd.read_csv("dataset.csv")  # CSV

import pandas as pd
df = pd.read_json("dataset.json")  # JSON

with open("dataset.txt", "r") as f: # TXT
    data = f.readlines()


In [ ]:
# Install necessary libraries
!pip install trl
!pip install unsloth

# Standard library imports
import os
import warnings

# Third-party library imports
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer, AutoTokenizer, AutoModelForSequenceClassification
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported
from datasets import Dataset

# Configure warnings and matplotlib
warnings.filterwarnings("ignore")
%matplotlib inline
plt.style.use('ggplot')

# Set device (GPU if available, otherwise CPU)
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

ModuleNotFoundError: No module named 'trl'

In [ ]:
# WE are using lama with 1B parameters.
# Max length set at 5020, max num of tokens the model can handle in a single input

max_seq_length = 1024  # imo its enough for a simple AI chatbot
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-1B-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True, # enables efficient 4-bit quantization
    dtype=None,
)
# we use parameter efficient fine tuning like we learnt in LLM which applied LORA techniques. This approach
# focuses on fine tuning only specific layers or parts of the model, rather than the entire network.
# r = 16 and lora_alpha = 16 adjusts the complexity and scaling of these adaptations.
#  target modules specifies which layers of the model should be adapted, which include key components involed
# in attention mechanisms like q_proj and k_proj and v_proj.
# use_rslora activates Rank stabalized LORA, which improves the stability of the fine tuning process.
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    # q_proj, k_proj, v_proj: Handle the query, key, and value projections in the attention mechanism, essential for capturing contextual information.
    # up_proj, down_proj: Layers in feedforward networks. o_proj: Combines attention heads’ output. gate_proj: Controls flow in certain feedforward networks.
    use_rslora=True,
    use_gradient_checkpointing="unsloth",
    random_state = 32,
    loftq_config = None,
)
print(model.print_trainable_parameters())

Sample Data For The Arol Group Chatbot.
This part will be replaced after we data scrap the websites later.

In [ ]:
sample_data = [
    {
        "Query": "What are the different types of bottle caps AROL Group manufactures?",
        "Response": "AROL Group manufactures various types of bottle caps, including screw caps, crown caps, and corks, designed for different industries and bottle types."
    },
    {
        "Query": "Can you explain how the capping machines work?",
        "Response": "Capping machines from AROL use precision automation to apply caps on bottles at high speed, ensuring a tight and consistent seal to prevent leakage and maintain product quality."
    },
    {
        "Query": "What industries does AROL Group provide solutions for?",
        "Response": "AROL Group provides capping and sealing solutions for industries such as food & beverage, personal care, pharmaceuticals, and chemical products."
    },
    {
        "Query": "How do AROL's machines ensure product safety?",
        "Response": "AROL machines use advanced inspection systems to detect any defects in capping, ensuring product safety by maintaining a consistent and contamination-free seal."
    },
    {
        "Query": "Does AROL offer maintenance support for their capping machines?",
        "Response": "Yes, AROL offers comprehensive maintenance support, including regular inspections, troubleshooting, and part replacements to keep machines running efficiently."
    },
    {
        "Query": "What are the environmental impacts of AROL's products?",
        "Response": "AROL is committed to sustainability, designing energy-efficient machines and encouraging the use of recyclable caps to minimize environmental impact."
    },
    {
        "Query": "Are AROL's machines customizable for different bottle sizes?",
        "Response": "Yes, AROL's capping machines are highly customizable to fit a wide range of bottle sizes, cap types, and production line configurations."
    },
    {
        "Query": "How does AROL ensure precision in high-speed bottling?",
        "Response": "AROL ensures precision with advanced robotics and servo-controlled systems that maintain accuracy and consistency, even at high production speeds."
    },
    {
        "Query": "What quality standards does AROL adhere to?",
        "Response": "AROL adheres to strict international quality standards, including ISO certifications, to ensure the reliability and safety of its products."
    },
    {
        "Query": "Does AROL offer training for operating their capping machines?",
        "Response": "Yes, AROL offers training programs to help operators understand and efficiently use their capping machines, covering safety, maintenance, and operational techniques."
    }
]


Now we convert the sample data to prompt template that we want to use for our chatbot.

In [ ]:
import pandas as pd

# first we convert sample data to a DataFrame
df = pd.DataFrame(sample_data)

# prompt template
data_prompt = """
You are a customer support assistant for AROL Group, specialized in bottle caps and capping technologies.
Your goal is to provide accurate, clear, and helpful responses to queries regarding our products, processes, and technologies.

### Input:
{query}

### Instructions:
- Provide a concise and informative response regarding bottle cap manufacturing or capping technology.
- If there are any technical details or product features mentioned, explain them in simple terms.
- If the query highlights any concerns, provide recommendations or solutions.
- Ensure the response is relevant to the specific question asked.

### Response:
{}
"""

EOS_TOKEN = tokenizer.eos_token  # Keep this for the end-of-sequence token


def formatting_prompt(df):
  inputs = df["Query"]
  outputs = df["Response"]
  texts = []
  for input_, output in zip(inputs, outputs):
    text = data_prompt.format(input_, output) + EOS_TOKEN
    texts.append(text)
  return {"text": texts,}

formatted_data = formatting_prompt(df)
# Convert formatted data into the form the model expects
print(formatted_data["text"][:2])  # Preview the first two formatted examples


In [ ]:
training_data = Dataset.from_pandas(df) # here we convert pandas dataframe into a hugging face dataset object
training_data = training_data.map(formatting_prompt, batched=True) # Here we apply the formatting func to each element of the dataset using map method.


Training the Model with the Trainer API
Goal: Use the Trainer API to actually fine-tune the model on the formatted dataset. This step leverages all previous configurations for efficient training.

Process:

The formatted data is fed into the Trainer as input for model training.
The Trainer uses LoRA fine-tuning to adjust only specific layers, optimizing performance while keeping memory usage low.
Purpose: This final step leverages all previous configurations and formatted data to train the model. The Trainer applies gradient updates to the specified layers according to LoRA parameters, optimizing the model for the task without requiring massive resources.

In [ ]:

trainer=SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=training_data,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args=TrainingArguments(
        learning_rate=3e-4,
        lr_scheduler_type="linear",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        num_train_epochs=40,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=10,
        output_dir="output",
        seed=0,
    ),
)

trainer.train()
